In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from numpy.linalg import inv, pinv
from sklearn.metrics import r2_score, mean_absolute_error

In [ ]:
file_path = 'D:\VSCODE\PyPrograms\ELM2\os-elm\data\nyc_taxi.csv'
url = 'https://raw.githubusercontent.com/alexeynau/os-elm/main/data/nyc_taxi.csv'
df = pd.read_csv(url)
df.head(5)

In [ ]:
stdSeq = float(np.std(df[['value']]))
meanSeq = float(np.mean(df['value']))
print(f'std: {stdSeq}, mean {meanSeq}')
df['z-scaled'] = StandardScaler().fit_transform(df[['value']])
df.head(5)

In [ ]:
class OSELM:
    def __init__(self, nInputs, nOutputs, numHiddenNeurons, forgettingFactor=1, gamma = 0.0001):

        self.numHiddenNeurons = numHiddenNeurons
        self.lamb = forgettingFactor

        # Веса и смещение выбираются случайным образом в интервале [-1; 1]
        self.alpha = np.random.random((self.numHiddenNeurons, nInputs)) * 2 - 1 
        self.bias = np.random.random((1, self.numHiddenNeurons)) * 2 - 1

        self.beta = np.zeros((self.numHiddenNeurons, nOutputs)) # Формула 9
        self.P = inv(gamma*np.eye(self.numHiddenNeurons)) # Формула 9


    def compute_h(self, features):       
        return self.sigmoid(features @ self.alpha.T  + self.bias)


    def sigmoid(self, features):
        return 1.0 / (1.0 + np.exp(-features))


    def fit(self, features, targets):
        features = np.array(features)
        targets = np.array(targets)
    
        H = self.compute_h(features)

        k = 1/self.lamb
        self.P = k * self.P - self.P @ H.T @ pinv(self.lamb**2 + self.lamb * H @ self.P @ H.T) @ H @ self.P # Формула 12

        self.beta = self.beta + self.P @ H.T @ (targets - H @ self.beta) # Формула 10
  
    def predict(self, features):
        H = self.compute_h(features)
        prediction = H @ self.beta # Формула 2
        return prediction

In [ ]:
predictions = []
targets = Y[:-1]
oselm = OSELM(100, 1, 100, forgettingFactor=0.996)
for i in range(len(df)-101):
    oselm.fit(X[i], Y[i])
    prediction = oselm.predict(X[i+1])

    predictions.append(prediction[0])